# BiLSTM et CRF

Dans ce notebook nous nous utilisons les champs aléatoires conditionnels (conditional random fields ou CRFs) qui sont généralement utilisés pour le named entityy recognition dans le but de prédire des séquences valides.
Par exemple, on ne devrait pas avoir de `B-PER` suivi directement par un autre `B-PER`.

Jusqu'à présent, noous avons supposé que les tags entre eux étaient indépendants. Lors de labelisage de séquence, cette hypothèse est inexacte. En effet, si nous savons que la probabilité que le mot précédent soit «I-LOC» est élevée, alors le mot actuel est moins susceptible de faire partie de la même entité («I-LOC»). CRF, par essence, tente de prédire la séquence la plus probable. Si vous voulez en savoir plus sur le CRF, en particulier dans le domaine de la classification des séquences, [cette série de vidéos](https://youtu.be/GF3iSJkgPbA) par Hugo Larochelle est très bien. Pour l'implémentation CRF, on utilisera le package [pytorch-crf](https://github.com/kmkurn/pytorch-crf).

In [ ]:
#from google.colab import drive
#drive.mount("/content/gdrive")

#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
!pip install torchtext==0.6.0
!pip install pytorch-crf

import time
import torch
from torch import nn
from torch.optim import Adam
from torchtext.data import Field, NestedField, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from torchtext.vocab import Vocab
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
from torchcrf import CRF
import numpy as np
import pandas as pd

import time
import random
import string
from itertools import chain

# Préparation des données

In [2]:
# pour la reproductibilité
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(lower = True) 
TAG = data.Field(unk_token = None) # les tags sont tous connus on a alors unk_token = None
CHAR_NESTING= Field(tokenize=list)
CHAR = NestedField(CHAR_NESTING) 

train_data, valid_data, test_data = data.TabularDataset.splits(
        path="data_ner/",
        train="train.csv",
        validation="valid.csv",
        test="test.csv", format='csv', skip_header=True,
        fields=(
            (("text", "char"), (TEXT, CHAR)), 
            ("tag", TAG)
        )
    )

MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ, # les mots qui apparaissent moins que MIN_FREQ fois seront ignorés du vocabulaire
                 vectors = "glove.6B.300d",
                 unk_init = torch.Tensor.normal_)


TAG.build_vocab(train_data)
CHAR.build_vocab(train_data) 
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort=False)

# padding index
TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
CHAR_PAD_IDX = CHAR.vocab.stoi[CHAR.pad_token]  
TAG_PAD_IDX = TAG.vocab.stoi[TAG.pad_token]

# Construire le modèle

Pour la classe de modèle, il existe trois mises à jour principales:

1. préparez la couche CRF lors de l'initialisation. Nous devons spécifier le nombre de tags possibles dans le text.
2. inclure la logique de la couche CRF dans la séquence `forward ()`. Il y a un changement majeur dans le `forward ()` en raison de l'implémentation du package `pytorch-crf`. Jusqu'à présent, nous effectuons séparément la propagation forward et le calcul des pertes. Désormais, le calcul des pertes est intégré à la propagation forward.
3. initialisez toutes les transitions impossibles avec un nombre vraiment bas (-100) dans la fonction `init_crf_transitions`. C'est là que nous imposons la logique de séquence BIO au modèle.

In [26]:
class BiLSTM(nn.Module):

    def __init__(self,
                 input_dim,
                 embedding_dim,
                 char_emb_dim,
                 char_input_dim,
                 char_cnn_filter_num,
                 char_cnn_kernel_size,
                 hidden_dim,
                 output_dim,
                 lstm_layers,
                 emb_dropout,
                 cnn_dropout,
                 lstm_dropout,
                 fc_dropout,
                 word_pad_idx,
                 char_pad_idx,
                 tag_pad_idx):
        super().__init__()
        
        # LAYER 1A: Word Embedding
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(
            num_embeddings=input_dim,
            embedding_dim=embedding_dim,
            padding_idx=word_pad_idx
        )
        self.emb_dropout = nn.Dropout(emb_dropout)
        
        # LAYER 1B: Char Embedding-CNN
        self.char_emb_dim = char_emb_dim
        self.char_emb = nn.Embedding(
            num_embeddings=char_input_dim,
            embedding_dim=char_emb_dim,
            padding_idx=char_pad_idx
        )
        self.char_cnn = nn.Conv1d(
            in_channels=char_emb_dim,
            out_channels=char_emb_dim * char_cnn_filter_num,
            kernel_size=char_cnn_kernel_size,
            groups=char_emb_dim  # different 1d conv for each embedding dim
        )
        self.cnn_dropout = nn.Dropout(cnn_dropout)
        
        # LAYER 2: BiLSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim + (char_emb_dim * char_cnn_filter_num),
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            bidirectional=True,
            dropout=lstm_dropout if lstm_layers > 1 else 0
        )
        
        # LAYER 3: Fully-connected  
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  
        # LAYER 4: CRF

        self.tag_pad_idx = tag_pad_idx
        self.crf = CRF(num_tags=output_dim)
        
        # init poids avec distribution normale 
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1)

    def forward(self, words, chars, tags=None):

        # words = [sentence length, batch size]
        # chars = [batch size, sentence length, word length)
        # tags = [sentence length, batch size]
        
        # embedding_out = [sentence length, batch size, embedding dim]
        embedding_out = self.emb_dropout(self.embedding(words))
    
        # char_emb_out = [batch size, sentence length, word length, char emb dim]
        char_emb_out = self.emb_dropout(self.char_emb(chars))
        
        batch_size, sent_len, word_len, char_emb_dim = char_emb_out.shape
        char_cnn_max_out = torch.zeros(batch_size, sent_len, self.char_cnn.out_channels)
        for sent_i in range(sent_len):
            
            # sent_char_emb = [batch size, word length, char emb dim]
            sent_char_emb = char_emb_out[:, sent_i, :, :]
            
            # sent_char_emb_p = [batch size, char emb dim, word length]
            sent_char_emb_p = sent_char_emb.permute(0, 2, 1)
            
            # char_cnn_sent_out = [batch size, out channels * char emb dim, word length - kernel size + 1]
            char_cnn_sent_out = self.char_cnn(sent_char_emb_p)
            char_cnn_max_out[:, sent_i, :], _ = torch.max(char_cnn_sent_out, dim=2)
        char_cnn = self.cnn_dropout(char_cnn_max_out)
        
        
        # char_cnn_p = [sentence length, batch size, char emb dim * num filter]
        char_cnn_p = char_cnn.permute(1, 0, 2).to(device)
        
        # concat word et char embedding
        word_features = torch.cat((embedding_out, char_cnn_p), dim=2)
        
        # lstm_out = [sentence length, batch size, hidden dim * 2]
        lstm_out, _ = self.lstm(word_features)
        
        # fc_out = [sentence length, batch size, output dim]
        fc_out = self.fc(self.fc_dropout(lstm_out))
        if tags is not None:
            mask = tags != self.tag_pad_idx
            crf_out = self.crf.decode(fc_out, mask=mask)
            crf_loss = -self.crf(fc_out, tags=tags, mask=mask) if tags is not None else None
        else:
            crf_out = self.crf.decode(fc_out)
            crf_loss = None
          
        return crf_out , crf_loss

    def init_embeddings(self, char_pad_idx, word_pad_idx, pretrained=None, freeze=True):
     
        # init embedding pad à zéro
        self.embedding.weight.data[word_pad_idx] = torch.zeros(self.embedding_dim)
        self.char_emb.weight.data[char_pad_idx] = torch.zeros(self.char_emb_dim)
        if pretrained is not None:
            self.embedding = nn.Embedding.from_pretrained(
                embeddings=torch.as_tensor(pretrained),
                padding_idx=word_pad_idx,
                freeze=freeze
            )

   
    ### BEGIN MODIFIED SECTION: CRF OUTPUT ###
    def init_crf_transitions(self, tag_names, imp_value=-100):
        num_tags = len(tag_names)
        for i in range(num_tags):
            tag_name = tag_names[i]
            # I and <pad> impossible as a start
            if tag_name[0] == "I" or tag_name == "<pad>":
                torch.nn.init.constant_(self.crf.start_transitions[i], imp_value)
        # init impossible transitions between positions
        tag_is = {}
        for tag_position in ("B", "I", "O"):
            tag_is[tag_position] = [i for i, tag in enumerate(tag_names) if tag[0] == tag_position]
        impossible_transitions_position = {
            "O": "I"

        }
        for from_tag, to_tag_list in impossible_transitions_position.items():
            to_tags = list(to_tag_list)
            
            for from_tag_i in tag_is[from_tag]:
                for to_tag in to_tags:
                    for to_tag_i in tag_is[to_tag]:
                    
                        torch.nn.init.constant_(
                            self.crf.transitions[from_tag_i, to_tag_i], imp_value
                        )
        # init impossible B and I transitions to different entity types
        impossible_transitions_tags = {
            "B": "I",
            "I": "I"
        }
        for from_tag, to_tag_list in impossible_transitions_tags.items():
            to_tags = list(to_tag_list)
            for from_tag_i in tag_is[from_tag]:
                for to_tag in to_tags:
                    for to_tag_i in tag_is[to_tag]:
                        if tag_names[from_tag_i].split("-")[1] != tag_names[to_tag_i].split("-")[1]:
                            torch.nn.init.constant_(
                                self.crf.transitions[from_tag_i, to_tag_i], imp_value
                            )

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [27]:
model = BiLSTM(
    input_dim=len(TEXT.vocab),
    embedding_dim=300,
    char_emb_dim=25,
    char_input_dim=len(CHAR.vocab),
    char_cnn_filter_num=5,
    char_cnn_kernel_size=2,
    hidden_dim=64,
    output_dim=len(TAG.vocab),
    lstm_layers=3,
    emb_dropout=0.5,
    cnn_dropout=0.25,
    lstm_dropout=0.1,
    fc_dropout=0.25,
    word_pad_idx=TEXT_PAD_IDX,
    char_pad_idx=CHAR_PAD_IDX,
    tag_pad_idx=TAG_PAD_IDX
)
model.init_embeddings(
    char_pad_idx=CHAR_PAD_IDX,
    word_pad_idx=TEXT_PAD_IDX,
    pretrained= TEXT.vocab.vectors
    ,freeze=True
)
# CRF transitions initialisation
model.init_crf_transitions(
    tag_names=TAG.vocab.itos
)
print(f"Le modèle a {model.count_parameters():,} paramètres à entraîner.")

Le modèle a 351,583 paramètres à entraîner.


Nous pouvons accéder à la matrice de transition et nous assurer que l'initialisation est effectuée comme prévu :

In [28]:
def print_crf_transitions(c, m):
    tags = TAG.vocab.itos
    max_len_tag = max([len(tag) for tag in tags])
    print("Start and end tag transitions:")
    print(f"{'TAG'.ljust(max_len_tag)}\tSTART\tEND")
    for tag, start_prob, end_prob in zip(tags, m.crf.start_transitions.tolist(), m.crf.end_transitions.tolist()):
        print(f"{tag.ljust(max_len_tag)}\t{round(start_prob, 2)}\t{round(end_prob, 2)}")
    print()
    print("Between tags transitions:")
    persons_i = [i for i, tag in enumerate(TAG.vocab.itos) if "PER" in tag or tag == "O"]
    max_len_tag = max([len(tag) for tag in TAG.vocab.itos if "PER" in tag ])
    transitions = m.crf.transitions
    to_tags = "TO".rjust(max_len_tag) + "\t" + "\t".join([tag.ljust(max_len_tag) for tag in tags if "PER" in tag or tag == "O"])
    print(to_tags)
    print("FROM")
    for from_tag_i, from_tag_probs in enumerate(transitions[persons_i]):
        to_tag_str = f"{tags[persons_i[from_tag_i]].ljust(max_len_tag)}"
        for to_tag_prob in from_tag_probs[persons_i]:
            to_tag_str += f"\t{str(round(to_tag_prob.item(), 2)).ljust(max_len_tag)}"
        print(to_tag_str)

print_crf_transitions(TEXT, model)

Start and end tag transitions:
TAG   	START	END
<pad> 	-100.0	-0.0
O     	0.01	-0.03
B-LOC 	0.06	0.11
B-PER 	-0.06	-0.06
B-ORG 	-0.12	0.07
I-PER 	-100.0	0.0
I-ORG 	-100.0	0.06
B-MISC	-0.15	-0.15
I-LOC 	-100.0	-0.1
I-MISC	-100.0	0.13

Between tags transitions:
   TO	O    	B-PER	I-PER
FROM
O    	-0.04	-0.03	-100.0
B-PER	0.02 	0.07 	-0.08
I-PER	0.0  	0.14 	-0.01


# Entraînement

Les sorties du modèle sont deux listes : les prédictions et les pertes.
On prend en considération ces changements en modifiant la fonction qui calcule l'accuracy.

 - Optimiseur et fonction loss

In [6]:
optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

model = model.to(device)
criterion = criterion.to(device)

 - Métriques

In [7]:
from sklearn.metrics import f1_score
def f1_loss(preds, y, tag_pad_idx):
    index_o = TAG.vocab.stoi["O"]
    positive_labels = [i for i in range(len(TAG.vocab.itos))
                           if i not in (tag_pad_idx, index_o)]
    flatten_preds = [pred for sent_pred in preds for pred in sent_pred]
    positive_preds = [pred for pred in flatten_preds
                          if pred not in (tag_pad_idx, index_o)]
    flatten_y = [tag for sent_tag in y for tag in sent_tag]
    
    f1 = f1_score(
            y_true=flatten_y,
            y_pred=flatten_preds,
            labels=positive_labels,
            average="micro"
        ) if len(positive_preds) > 0 else 0
       
    return f1

def accuracy_per_tag(predictions, tags):
    n_tags = len(TAG.vocab)
    class_correct = list(0 for i in range(n_tags))
    class_total = list(0 for i in range(n_tags))
    acc = list(0 for i in range(n_tags))
    
    flatten_preds = [pred for sent_pred in predictions for pred in sent_pred]

    flatten_y = [tag for sent_tag in tags for tag in sent_tag]

    # # compare predictions to true label
    correct = [pred == tag for pred, tag in zip(flatten_preds, flatten_y)]

    # # calculate test accuracy for each object class
    for i in range(len(flatten_y)):
        label = flatten_y[i]
        class_correct[label] += correct[i]
        class_total[label] += 1
    for i in range(n_tags):
        if np.sum(class_total[i]) == 0 and np.sum(class_correct[i]) == 0:
            res = 100
        else:
            res = 100 * class_correct[i] / class_total[i]
        acc[i] = res, np.sum(class_correct[i]), np.sum(class_total[i])
        
    return acc  

In [8]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.tag
        chars = batch.char 
        optimizer.zero_grad()

        pred_tags_list, batch_loss = model(text, chars, tags)
        
        # pour calculer la loss et le score f1, on flatten true tags
        true_tags_list = [
                [tag for tag in sent_tag if tag != TAG_PAD_IDX]
                for sent_tag in tags.permute(1, 0).tolist()
            ]
        f1 = f1_loss(pred_tags_list, true_tags_list, tag_pad_idx)

        acc = accuracy_per_tag(pred_tags_list, true_tags_list)
        batch_loss.backward()
        
        optimizer.step()
        epoch_loss += batch_loss.item()
        epoch_f1 += f1
        
    return epoch_loss / len(iterator), acc, epoch_f1 / len(iterator)

def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.tag
            chars = batch.char
            
            pred_tags_list, batch_loss = model(text, chars, tags)
            true_tags_list = [
                [tag for tag in sent_tag if tag != TAG_PAD_IDX]
                for sent_tag in tags.permute(1, 0).tolist()
                ]
            
            f1 = f1_loss(pred_tags_list, true_tags_list, tag_pad_idx)
            acc = accuracy_per_tag(pred_tags_list, true_tags_list)   
            epoch_loss += batch_loss.item()
            epoch_f1 += f1
        
    return epoch_loss / len(iterator), acc, epoch_f1 / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc, valid_f1 = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)

    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    if epoch%10 == 0:
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train F1 score: {train_f1*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} | Val. F1 score: {valid_f1*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 4s
	Train Loss: 1012.668 | Train F1 score: 6.73%
	 Val. Loss: 622.206 | Val. F1 score: 31.95%
Epoch: 06 | Epoch Time: 0m 57s
	Train Loss: 146.290 | Train F1 score: 82.95%
	 Val. Loss: 125.085 | Val. F1 score: 83.14%
Epoch: 11 | Epoch Time: 1m 1s
	Train Loss: 99.817 | Train F1 score: 87.85%
	 Val. Loss: 91.937 | Val. F1 score: 89.02%
Epoch: 16 | Epoch Time: 1m 45s
	Train Loss: 80.490 | Train F1 score: 90.06%
	 Val. Loss: 83.835 | Val. F1 score: 89.71%
Epoch: 21 | Epoch Time: 3m 5s
	Train Loss: 67.804 | Train F1 score: 91.42%
	 Val. Loss: 84.122 | Val. F1 score: 89.24%
Epoch: 26 | Epoch Time: 3m 6s
	Train Loss: 61.484 | Train F1 score: 92.27%
	 Val. Loss: 80.855 | Val. F1 score: 90.04%


In [9]:
n_tags = len(TAG.vocab)
for i in range(n_tags):   
    print('Train Accuracy de %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], train_acc[i][0],
           train_acc[i][1], train_acc[i][2]))  

Train Accuracy of <pad>: 100% ( 0/ 0)
Train Accuracy of     O: 99% (1433/1439)
Train Accuracy of B-LOC: 94% (51/54)
Train Accuracy of B-PER: 100% (54/54)
Train Accuracy of B-ORG: 94% (55/58)
Train Accuracy of I-PER: 100% (32/32)
Train Accuracy of I-ORG: 100% (31/31)
Train Accuracy of B-MISC: 89% (26/29)
Train Accuracy of I-LOC: 100% (12/12)
Train Accuracy of I-MISC: 77% ( 7/ 9)


In [10]:
for i in range(n_tags):
    print('Valid Accuracy de %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], valid_acc[i][0],
           valid_acc[i][1], valid_acc[i][2]))
  

Valid Accuracy of <pad>: 100% ( 0/ 0)
Valid Accuracy of     O: 100% (148/148)
Valid Accuracy of B-LOC: 100% ( 1/ 1)
Valid Accuracy of B-PER: 100% ( 2/ 2)
Valid Accuracy of B-ORG: 71% ( 5/ 7)
Valid Accuracy of I-PER: 100% ( 2/ 2)
Valid Accuracy of I-ORG: 60% ( 3/ 5)
Valid Accuracy of B-MISC: 100% ( 1/ 1)
Valid Accuracy of I-LOC: 100% ( 0/ 0)
Valid Accuracy of I-MISC: 100% ( 0/ 0)


In [11]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc, test_f1 = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)
n_tags = len(TAG.vocab)
for i in range(n_tags):   
    print('Test Accuracy de %5s: %2d%% (%2d/%2d)' % (
           TAG.vocab.itos[i], test_acc[i][0],
           test_acc[i][1], test_acc[i][2]))
print(f'Test Loss: {test_loss:.3f} |  Test F1 score: {test_f1*100:.2f}%')

Test Accuracy of <pad>: 100% ( 0/ 0)
Test Accuracy of     O: 99% (909/918)
Test Accuracy of B-LOC: 79% (27/34)
Test Accuracy of B-PER: 66% (24/36)
Test Accuracy of B-ORG: 85% (68/80)
Test Accuracy of I-PER: 100% (14/14)
Test Accuracy of I-ORG: 75% (22/29)
Test Accuracy of B-MISC: 85% (18/21)
Test Accuracy of I-LOC: 87% ( 7/ 8)
Test Accuracy of I-MISC: 100% ( 2/ 2)
Test Loss: 130.054 |  Test F1 score: 85.73%


## Inférence

In [12]:
def tag_sentence(model, device, sentence, text_field, tag_field, char_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    max_word_len = max([len(token) for token in tokens])
    numericalized_chars = []
    char_pad_id = char_field.vocab.stoi[CHAR.pad_token] 
    for token in tokens:
        numericalized_chars.append(
                [char_field.vocab.stoi[char] for char in token]
                + [char_pad_id for _ in range(max_word_len - len(token))]
                )
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]
    unk_idx = text_field.vocab.stoi[text_field.unk_token]  
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.as_tensor(numericalized_tokens)    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
    char_tensor = torch.as_tensor(numericalized_chars)
    char_tensor = char_tensor.unsqueeze(0).to(device) 
    predictions, _ = model(token_tensor, char_tensor)
    print(predictions)
    predicted_tags = [tag_field.vocab.itos[t] for t in predictions[0]]
    
    return tokens, predicted_tags, unks

In [31]:
example_index = 1

sentence = vars(valid_data.examples[example_index])['text']
actual_tags = vars(valid_data.examples[example_index])['tag']

print(sentence)
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       TEXT, 
                                       TAG,
                                       CHAR)

print(unks)
print(pred_tags)

['cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.']


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select

In [32]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

O		O		✔		cricket
O		O		✔		-
O		B-ORG		✘		leicestershire
O		O		✔		take
O		O		✔		over
O		O		✔		at
O		O		✔		top
O		O		✔		after
O		O		✔		innings
O		O		✔		victory
O		O		✔		.


In [33]:
sentence = 'The will deliver a speech about the conflict in Sao Paulo at tomorrow in Anne Mary with Jack.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                  TAG,
                                  CHAR)

print(unks)
print("Pred. Tag\tToken\n")


for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select